In [1]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set(rc={'figure.figsize': (25, 10)}, font_scale=1.5)

df = pd.read_csv('/content/drive/MyDrive/ECH_2019.csv')

# df = pd.read_csv('ECH_2019.csv')

In [3]:
df.sample(2)



,id_hogar,sexo,edad,ascendencia,parentesco,estado_laboral,sueldo,busca_otro_trabajo,actividad,asistencia alguna vez a enseñanza,edu preescolar,edu primaria,edu media,edu técnica,edu univ o simil,edu terciario no uni,edu posgrado,edu mag prof
36072,2019019434,Hombre,18,Blanca,Hijo/a de ambos,Inactivo: otro,0,0,.,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",No asistió,0,0,0,0
36061,2019019426,Hombre,21,Blanca,Hijo/a solo del jefe/a,Ocupados,20000,Sí,Cultivo de frutas cítricas,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",No asistió,0,0,0,0


In [4]:
no_data = {0: 'No hay dato', '0': 'No hay dato'}

mask = df.columns.difference(['edad', 'sueldo', 'hijos'])

df[mask] = df[mask].replace(no_data)

df.sample(2)



,id_hogar,sexo,edad,ascendencia,parentesco,estado_laboral,sueldo,busca_otro_trabajo,actividad,asistencia alguna vez a enseñanza,edu preescolar,edu primaria,edu media,edu técnica,edu univ o simil,edu terciario no uni,edu posgrado,edu mag prof
9665,2019005178,Mujer,23,Blanca,Hijo/a solo del jefe/a,Ocupados,10000,No,Actividades de peluquería y otros tratamientos...,Sí,"Sí, asistió","Sí, asistió","Sí, asistió","Sí, asistió",No asistió,No asistió,No hay dato,No asistió
95387,2019051339,Hombre,88,Blanca,Jefe/a de hogar,Inactivo: jubilado,0,No hay dato,.,Sí,No asistió,"Sí, asistió",No asistió,No asistió,No hay dato,No hay dato,No hay dato,No hay dato


Seleccionamos personas con edad suficiente para trabajar

In [5]:
# df = df[~df['estado_laboral'].str.contains('Menores de 14 años')]



Creamos una columna que cuenta la cantidad de hijos por grupo familiar

In [6]:
df['hijos'] = (
    df.groupby('id_hogar')['id_hogar'].transform('count') - 1
)



# CATEGORIZAR NUMERICOS

In [7]:
df['rango_sueldos'] = pd.cut(df['sueldo'], bins=[
    0,
    5000,
    10000,
    30000,
    50000,
    100000,
    float('Inf'),
    ]).astype('category')

df['rango_edades'] = pd.cut(df['edad'], bins=[
    0,
    10,
    15,
    20,
    30,
    40,
    50,
    60,
    70,
    float('Inf'),
    ]).astype('category')



### AJUSTANDO DATOS PARA ENTRENAR EL MODELO

Primero encodificamos las variables categóricas

Eliminanos las columnas que no vamos a usar o que ya categorizamos

In [8]:
df = df[[
    'sexo',
    'ascendencia',
    'estado_laboral',
    'busca_otro_trabajo',
    'actividad',
    'asistencia alguna vez a enseñanza',
    'edu preescolar',
    'edu primaria',
    'edu media',
    'edu técnica',
    'edu univ o simil',
    'edu terciario no uni',
    'edu posgrado',
    'edu mag prof',
    'parentesco',
    'hijos',
    'rango_sueldos',
    'rango_edades',
    ]]

    

In [9]:
df['estado_laboral'].unique()



array(['Inactivo: jubilado', 'Inactivo: realiza los quehaceres del hogar',
       'Ocupados', 'Menores de 14 años', 'Inactivo: estudiante',
       'Desocupados propiamente dichos', 'Inactivo: pensionista',
       'Inactivo: otro', 'Desocupados en seguro de paro',
       'Desocupados buscan trabajo por primera vez', 'Inactivo: rentista'],
      dtype=object)

## Reducimos la consideración de estado laboral a menos categorías

In [10]:
def change_cat(value):
    if 'Desocupados' in value:
        return 'Desocupado'
    if 'Inactivo' in value:
        return 'Inactivo'
    if 'Menores' in value:
        return 'Menor'
    else:
        return 'Ocupado'

df['estado_laboral'] = df['estado_laboral'].map(change_cat)

df['estado_laboral'].unique()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


array(['Inactivo', 'Ocupado', 'Menor', 'Desocupado'], dtype=object)

### DEFINIMOS LAS VARIABLES DEPENDIENTES E INDEPENDIENTES

In [11]:
# Feautures
X = df.drop('estado_laboral', axis=1).to_numpy()
y = df['estado_laboral']

print('X shape', X.shape, '\nY shape', y.shape)



X shape (107871, 17) 
Y shape (107871,)


In [12]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()

X = one_hot.fit_transform(X)



### Dividimos los datos en un 30% de test y 70% de entrenamiento

In [13]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 



In [14]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



(75509, 477) (75509,)
(32362, 477) (32362,)


# DEFINIMOS ALGORITMO DE REGRESIÓN LOGISTICA

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logistic_reg = LogisticRegression(multi_class='multinomial', max_iter=20000)

logistic_reg.fit(X_train, y_train)



### VALOR DE CROSS VALIDATION PARA CADA FOLD

In [16]:
scores = cross_val_score(logistic_reg, X, y, cv=10)

scores



array([0.94670004, 0.94706591, 0.94808566, 0.94595346, 0.94836377,
       0.94827107, 0.94929081, 0.94780755, 0.9490127 , 0.94660239])

In [17]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



Promedio de cross_validation  0.95
Score en test  0.95
Score en train  0.95


In [18]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predictions))



[[  269  1021     0     0]
 [  185 10258   442     0]
 [    0    35  5380     0]
 [    0     0     0 14772]]


### INVESTIGAMOS LA EFECTIVIDAD DEL MODELO SOBRE LOS DATOS

In [19]:
from sklearn.metrics import accuracy_score

#Prediccion en Train
y_train_pred = logistic_reg.predict(X_train)

#Prediccion en Test
y_test_pred = logistic_reg.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



% de aciertos sobre el set de entrenamiento: 0.9484962057503079
% de aciertos sobre el set de evaluación: 0.9479945615227736


### DEFINIMOS UNA PREDICCIÓN Y LA APLICAMOS SOBRE EL MODELO

In [20]:
prediction = X.toarray()

prediction = prediction[1000].reshape(1, -1)

print(logistic_reg.predict(prediction))
print(y_train[1000])



['Inactivo']
Inactivo


Determinamos la probabilidad de la predicción

In [21]:
probabilidades_prediccion = logistic_reg.predict_proba(prediction)
probabilidades_prediccion[:,1]



array([0.85013448])

# REPETIMOS EL EJERCICIO CON EL MODELO DE DECISSION TREE

In [22]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(100)

random_forest.fit(X_train, y_train)



In [23]:
scores = cross_val_score(random_forest, X, y, cv=10)

scores



array([0.94456804, 0.94335775, 0.94252341, 0.9424307 , 0.94326504,
       0.94511912, 0.94660239, 0.9422453 , 0.94502642, 0.94409938])

In [24]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



Promedio de cross_validation  0.94
Score en test  0.95
Score en train  0.95


In [25]:
#Prediccion en Train
y_train_pred = random_forest.predict(X_train)

#Prediccion en Test
y_test_pred = random_forest.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



% de aciertos sobre el set de entrenamiento: 0.9638321259717385
% de aciertos sobre el set de evaluación: 0.9440084049193499


In [26]:
print(random_forest.predict(prediction))
print(y_train[1000])



['Inactivo']
Inactivo


\# REPETIMOS EL EJERCICIO CON EL MODELO DE SUPPORT VECTOR MACHINE

In [27]:
from sklearn import svm

svm_model = svm.SVC(kernel='linear')

svm_model.fit(X_train, y_train)



array(['Menor', 'Ocupado', 'Inactivo', ..., 'Inactivo', 'Ocupado',
       'Inactivo'], dtype=object)

In [28]:
scores = cross_val_score(svm_model, X, y, cv=10)

scores



array([0.94503152, 0.9466951 , 0.9467878 , 0.94521183, 0.94632428,
       0.94780755, 0.94632428, 0.94613887, 0.94697321, 0.94539724])

In [29]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(svm_model.score(X_test, y_test),2))
print('Score en train ', round(svm_model.score(X_train, y_train),2))



Promedio de cross_validation  0.95
Score en test  0.95
Score en train  0.95


In [30]:
#Prediccion en Train
y_train_pred = svm_model.predict(X_train)

#Prediccion en Test
y_test_pred = svm_model.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



% de aciertos sobre el set de entrenamiento: 0.9462315750440344
% de aciertos sobre el set de evaluación: 0.9472220505531178


In [31]:
print(svm_model.predict(prediction))
print(y_train[1000])



['Inactivo']
Inactivo


\# REPETIMOS EL EJERCICIO CON EL MODELO DE K Nearest neighbours

In [32]:
from sklearn.neighbors import KNeighborsClassifier

knearest_model = KNeighborsClassifier(n_neighbors=4)

knearest_model.fit(X_train, y_train)

array(['Menor', 'Ocupado', 'Inactivo', ..., 'Inactivo', 'Ocupado',
       'Desocupado'], dtype=object)

In [33]:
scores = cross_val_score(knearest_model, X, y, cv=10)

scores

array([0.93242492, 0.92648558, 0.92787615, 0.93028646, 0.9334384 ,
       0.9268564 , 0.93408733, 0.92750533, 0.93204784, 0.9267637 ])

In [34]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(knearest_model.score(X_test, y_test),2))
print('Score en train ', round(knearest_model.score(X_train, y_train),2))



Promedio de cross_validation  0.93
Score en test  0.93
Score en train  0.95


In [35]:
#Prediccion en Train
y_train_pred = knearest_model.predict(X_train)

#Prediccion en Test
y_test_pred = knearest_model.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



% de aciertos sobre el set de entrenamiento: 0.9460726535909627
% de aciertos sobre el set de evaluación: 0.9311229219454916


In [36]:
print(knearest_model.predict(prediction))
print(y_train[1000])



['Inactivo']
Inactivo
